In [52]:
import os

In [53]:
from datasets import load_dataset
from datasets import Dataset
import numpy as np
from PIL import Image
from torchvision.transforms import ColorJitter
from transformers import SegformerFeatureExtractor
import json

In [54]:
data_split = ['train','val']

path = '/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation/'

In [55]:
fl = open('label2id.json')
label2id = json.load(fl)
fl.close()

In [56]:
fl = open('id2label.json')
id2label = json.load(fl)
fl.close()

In [57]:
def populate(cpath):
    
    result = []
    
    folders = os.listdir(cpath)
    
    for each_folder in folders:
        
        combined_folder_path = cpath+'/'+each_folder

        files = os.listdir(combined_folder_path)
        
        for each in files:
            
            result.append(combined_folder_path+'/'+each)
    
    return result

In [58]:
def get_all_images_path(split,subset=0):
    
    combined_path = path+'leftImg8bit/'+split+'/'
    
    images = sorted(populate(combined_path))
    
    combined_path = path+'label_processed/'+split+'/'
    
    masks = sorted(populate(combined_path))
    
    if subset>0:
        images = images[:subset]
        masks = masks[:subset]
    
    data_dict = {
        "pixel_values" : images,
        "label" : masks
    }
    
    return data_dict
    

    

In [59]:
d_train =  get_all_images_path('train',1000)
d_val = get_all_images_path('val',200)
# Create a Hugging Face Dataset from the dictionary
dataset_train = Dataset.from_dict(d_train)
dataset_val = Dataset.from_dict(d_val)

In [60]:
def mapping_fn(example):
    result = {}
    result['pixel_values'] = Image.open(example['pixel_values']).convert('RGB')
    result['label'] = Image.open(example['label']).convert('L')
    return result

In [62]:


feature_extractor = SegformerFeatureExtractor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 

def train_transforms(example_batch):
    images = [jitter(Image.open(x).convert('RGB')) for x in example_batch['pixel_values']]
    labels = [Image.open(x).convert('L') for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [Image.open(x).convert('RGB') for x in example_batch['pixel_values']]
    labels = [Image.open(x).convert('L') for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


# Set transforms
dataset_train.set_transform(train_transforms)
dataset_val.set_transform(val_transforms)

In [63]:
dataset_train[0]['labels']

array([[31, 31, 31, ..., 31, 31, 31],
       [31, 31, 31, ..., 31, 31, 31],
       [31, 31, 31, ..., 31, 31, 31],
       ...,
       [22, 22, 22, ...,  2,  2,  2],
       [22, 22, 22, ...,  2,  2,  2],
       [22, 22, 22, ...,  2,  2,  2]])

In [64]:
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "nvidia/mit-b0" 
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)


Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_fuse.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.2.proj.bias', 'decode_h

In [65]:
from transformers import TrainingArguments

epochs = 50
lr = 0.00006
batch_size = 2


training_args = TrainingArguments(
    "segformer-b0-finetuned-segments-sidewalk-outputs",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True

)


In [66]:
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    # currently using _compute instead of compute
    # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=feature_extractor.do_reduce_labels,
        )
    
    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[str(i)]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[str(i)]}": v for i, v in enumerate(per_category_iou)})
    
    return metrics


In [67]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

/gpfs/fs1/home/j/jcaunedo/umar1/nnlp/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Curb,Accuracy Caravan,Accuracy Road,Accuracy Out of roi,Accuracy Tunnel,Accuracy License plate,Accuracy Sidewalk,Accuracy Bus,Accuracy Trailer,Accuracy Vehicle fallback,Accuracy Obs-str-bar-fallback,Accuracy Autorickshaw,Accuracy Polegroup,Accuracy Animal,Accuracy Car,Accuracy Building,Accuracy Drivable fallback,Accuracy Train,Accuracy Rectification border,Accuracy Pole,Accuracy Ego vehicle,Accuracy Non-drivable fallback,Accuracy Motorcycle,Accuracy Fallback background,Accuracy Billboard,Accuracy Wall,Accuracy Parking,Accuracy Rider,Accuracy Rail track,Accuracy Traffic sign,Accuracy Guard rail,Accuracy Sky,Accuracy Fence,Accuracy Traffic light,Accuracy Bridge,Accuracy Person,Accuracy Bicycle,Accuracy Vegetation,Accuracy Truck,Iou Curb,Iou Caravan,Iou Road,Iou Out of roi,Iou Tunnel,Iou License plate,Iou Sidewalk,Iou Bus,Iou Trailer,Iou Vehicle fallback,Iou Obs-str-bar-fallback,Iou Autorickshaw,Iou Polegroup,Iou Animal,Iou Car,Iou Building,Iou Drivable fallback,Iou Train,Iou Rectification border,Iou Pole,Iou Ego vehicle,Iou Non-drivable fallback,Iou Motorcycle,Iou Fallback background,Iou Billboard,Iou Wall,Iou Parking,Iou Rider,Iou Rail track,Iou Traffic sign,Iou Guard rail,Iou Sky,Iou Fence,Iou Traffic light,Iou Bridge,Iou Person,Iou Bicycle,Iou Vegetation,Iou Truck
20,3.104600,3.319595,0.068692,0.145780,0.629247,nan,0.000000,0.858256,nan,nan,nan,0.059471,0.002642,nan,0.000000,0.094687,0.139915,0.000000,0.000000,0.774963,0.212985,0.042682,nan,nan,0.000649,nan,0.000679,0.011637,0.000000,0.006853,0.026912,nan,0.001425,nan,0.000000,0.001253,0.734803,0.000000,0.000000,0.000000,0.011790,0.268755,0.967694,0.009581,0.000000,0.000000,0.785948,0.000000,0.000000,0.000000,0.040557,0.002443,0.000000,0.000000,0.057372,0.050504,0.000000,0.000000,0.288023,0.151072,0.038429,0.000000,0.000000,0.000512,0.000000,0.000676,0.009673,0.000000,0.006656,0.019779,0.000000,0.001416,0.000000,0.000000,0.000903,0.726635,0.000000,0.000000,0.000000,0.009409,0.000148,0.479354,0.009464
40,2.384000,2.228946,0.080715,0.151299,0.708144,nan,0.000000,0.938151,nan,nan,nan,0.001769,0.001566,nan,0.000000,0.126907,0.000923,0.000000,0.000000,0.878234,0.543514,0.003226,nan,nan,0.000000,nan,0.000000,0.003685,0.000000,0.006559,0.003198,nan,0.000000,nan,0.000000,0.000000,0.916363,0.000000,0.000000,0.000000,0.000000,0.000000,0.962076,0.001507,0.000000,0.000000,0.837790,0.000000,nan,0.000000,0.001753,0.001549,nan,0.000000,0.068841,0.000898,0.000000,0.000000,0.215990,0.263016,0.003214,0.000000,0.000000,0.000000,nan,0.000000,0.003629,0.000000,0.006496,0.002970,0.000000,0.000000,0.000000,0.000000,0.000000,0.863155,0.000000,0.000000,0.000000,0.000000,0.000000,0.634927,0.001500
60,1.915300,1.738569,0.097080,0.157010,0.722363,nan,0.000000,0.948476,nan,nan,nan,0.008551,0.000075,nan,0.000000,0.042150,0.000000,0.000000,0.000000,0.893346,0.686104,0.037382,nan,nan,0.000000,nan,0.000000,0.000267,0.000000,0.000326,0.032097,nan,0.000000,nan,0.000000,0.000000,0.951521,0.000000,0.000000,0.000000,0.000000,0.000000,0.952551,0.000431,0.000000,0.000000,0.858397,nan,nan,nan,0.008423,0.000075,nan,0.000000,0.033825,0.000000,0.000000,0.000000,0.238424,0.239243,0.035316,0.000000,nan,0.000000,nan,0.000000,0.000267,0.000000,0.000326,0.021513,nan,0.000000,nan,0.000000,0.000000,0.881658,0.000000,0.000000,0.000000,0.000000,0.000000,0.691580,0.000430
80,2.451300,1.529708,0.103840,0.159235,0.733267,nan,0.000000,0.970947,nan,nan,nan,0.000802,0.000105,nan,0.000000,0.041008,0.000047,0.000000,0.000000,0.845162,0.731828,0.072458,nan,nan,0.000000,nan,0.000000,0.005856,0.000000,0.000246,0.038066,nan,0.000002,nan,0.000000,0.000000,0.966301,0.000000,0.000000,0.000000,0.000000,0.000000,0.944250,0.000741,0.000000,0.000000,0.864910,nan,nan,nan,0.000801,0.000105,nan,0.000000,0.032414,0.000047,0.000000,0.000000,0.304003,0.222926,0.062533,nan,nan,0.000000,nan,0.000000,0.005824,0.000000,0.000245,0.027960,nan,0.000002,nan,0.000000,0.000000,0.

/home/j/jcaunedo/umar1/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/home/j/jcaunedo/umar1/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/j/jcaunedo/umar1/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/home/j/jcaunedo/umar1/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d0532

In [48]:
id2label

{'0': 'curb',
 '1': 'caravan',
 '2': 'road',
 '3': 'out of roi',
 '4': 'tunnel',
 '5': 'license plate',
 '6': 'sidewalk',
 '7': 'bus',
 '8': 'trailer',
 '9': 'vehicle fallback',
 '10': 'obs-str-bar-fallback',
 '11': 'autorickshaw',
 '12': 'polegroup',
 '13': 'animal',
 '14': 'car',
 '15': 'building',
 '16': 'drivable fallback',
 '17': 'train',
 '18': 'rectification border',
 '19': 'pole',
 '20': 'ego vehicle',
 '21': 'non-drivable fallback',
 '22': 'motorcycle',
 '23': 'fallback background',
 '24': 'billboard',
 '25': 'wall',
 '26': 'parking',
 '27': 'rider',
 '28': 'rail track',
 '29': 'traffic sign',
 '30': 'guard rail',
 '31': 'sky',
 '32': 'fence',
 '33': 'traffic light',
 '34': 'bridge',
 '35': 'person',
 '36': 'bicycle',
 '37': 'vegetation',
 '38': 'truck'}